In this miniproject, we will combine and practice topics that we have learned in previous two weeks:

APIs
Databases (SQL)
Pandas
Processing special data types in Python
We will work with these APIs:

Foursquare - we have already come across this one
Yelp - this API offers similar services as Foursquare.
(Stretch) Google Places API - this google api offers similar service as well.
The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area. The project consists of following tasks:

pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area using these instructions). If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts, not the actual number of places we were able to get.

create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've created sqlite3 databases before in the activity Pandas with SQL.

compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.
choose the top 10 POIs based on the popularity (number of reviews or average rating) (Yelp, Foursquare).

(Stretch) By implementing travelling salesman problem (TSP), how much time would it take to visit all of these places? (Directions API from google will be helpful here). We will have to find travel time between all places (top 10). We can use ortools from Google to effectively implement TSP. These tools are very powerful and easy to install.


In [6]:
import requests
import pandas as pd
import numpy as np
import json
import os

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# Yelp API information
yelp_client_ID = os.getenv('yelp_client_ID')
yelp_api_key = os.getenv('yelp_api_key')
    
# Google API information
g_api_key = os.getenv('G_API_KEY')

# Foursquare API information
foursquare_id = os.getenv('foursquare_id')
foursquare_secret = os.getenv('foursquare_secret')

In [12]:
g_api_key

## Defining inputs
##### This block will take the inputs for place to search and radius to be used in other searches

In [338]:
place = input("Where would you like to search? ")

while True:
    try:
        rad = int(input("What search radius would you like to use (m)? "))
    except ValueError:
        print("Please input an integer")
        continue
    else:
        break


Where would you like to search?  Kimberley
What search radius would you like to use (m)?  2000


## Google API Request 
##### This block will take the input from the input block to return lat/long

In [358]:
# for location written down, give back lat,long coordinates

g_url = ("https://maps.googleapis.com/maps/api/place/findplacefromtext/"
    "json?"
    "input={1}"
    "&inputtype=textquery"
    "&fields=geometry"
    "&key={0}"
    .format(g_api_key, place))

g_resp = requests.get(g_url)

g_data = g_resp.json()

g_resp.status_code

200

In [360]:
g_data

{'candidates': [],
 'error_message': 'The provided API key is invalid.',
 'status': 'REQUEST_DENIED'}

In [333]:
coord_dict = {'lat':[],'long':[]}

lat = g_data['candidates'][0]['geometry']['location']['lat']
long = g_data['candidates'][0]['geometry']['location']['lng']

# print(lat)
# print(long)

IndexError: list index out of range

# Yelp API and Parse

In [267]:
# Yelp API request, this request sorts the results by rating

url = ("https://api.yelp.com/v3/businesses/search?"
    "term=&latitude={0}"
    "&longitude={1}"
    "&radius={2}"
    "&sort_by=rating"
    .format(lat,long,rad))

payload = {}
headers = {
  'Authorization': 'Bearer %s' % yelp_api_key
}

yelp_resp = requests.request("GET", url, headers=headers, data = payload)
yelp_data = yelp_resp.json()
yelp_resp.status_code

200

In [307]:
# initialize dictionary
yelp_dict = {'name':[],'latitude':[],'longitude':[]}

#parse through to add key information to dictionary    
for x in range(len(yelp_data['businesses'])):
    
    yelp_dict['name'].append(yelp_data['businesses'][x]['name'])
    yelp_dict['latitude'].append(yelp_data['businesses'][x]['coordinates']['latitude'])
    yelp_dict['longitude'].append(yelp_data['businesses'][x]['coordinates']['longitude'])

df_yelp = pd.DataFrame(yelp_dict)
df_yelp_10 = df_yelp.head(10)

In [308]:
print("According to Yelp, here are the top 10 places within {} m:".format(rad))
print(df_yelp_10['name'])

According to Yelp, here are the top 10 places within 1000 m:
0                Ten Foot Henry
1             Village Ice Cream
2                    Veg-In YYC
3                 OEB Breakfast
4    Workshop Kitchen + Culture
5                    ZCREW Cafe
6       Thi Thi Vietnamese Subs
7               Boogies Burgers
8                  Klein Harris
9           Charcut Roast House
Name: name, dtype: object


In [259]:
df_yelp.shape

(20, 5)

# Foursquare API and Parse

In [296]:
# Foursquare API request, this request sorts results by popularity

fs_url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id=foursquare_id,
client_secret=foursquare_secret,
v='20180323',
ll= str(lat)+','+str(long),
radius=rad,
section='topPicks',
sortByPopularity=1,
limit=200
)
fs_resp = requests.get(url=fs_url, params=params)
fs_data = json.loads(fs_resp.text)

fs_resp.status_code

200

In [300]:
#initialize dictionary
fs_dict = {'name':[],'latitude':[],'longitude':[]}

# parse through JSON for applicable data
for i in range(len(fs_data['response']['groups'])):
    for j in range(len(fs_data['response']['groups'][i]['items'])):
        fs_dict['name'].append(fs_data['response']['groups'][i]['items'][j]['venue']['name'])
        fs_dict['latitude'].append(fs_data['response']['groups'][i]['items'][j]['venue']['location']['lat'])
        fs_dict['longitude'].append(fs_data['response']['groups'][i]['items'][j]['venue']['location']['lng']) 

df_fs = pd.DataFrame(fs_dict)
df_fs_10 = df_fs.head(10)


In [298]:
df_fs_10

,id,name,latitude,longitude
0,52b3866711d2ccda9e35f954,"5 Vines Wine, Craft Beer & Spirits",51.041360,-114.059326
1,4d89722bf607a093a21eeb86,RiverWalk,51.047774,-114.049147
2,4bdc574efed22d7fcab657c9,Sien Lok Park,51.052199,-114.065061
3,4b6b6ebef964a52070082ce3,Calgary Court Restaurant,51.051128,-114.061454
4,4fe61052e4b05d1edc0100a5,Village Ice Cream,51.042734,-114.054295
5,4aeccc23f964a52049cb21e3,Bottlescrew Bill's Old English Pub,51.043545,-114.065531
6,518bfd76498e9a6d2a967141,Local 8th Avenue,51.045820,-114.068583
7,566f9b18498eafd7d1193e2d,Cactus Club Cafe,51.045966,-114.070166
8,4b0586eaf964a520367522e3,Bookers BBQ Grill & Crab Shack,51.049449,-114.055084
9,4fbb978fe4b0450d1d488e52,River Walk,51.051413,-114.059962


In [271]:
df_fs.shape

(100, 4)

In [ ]:
for row in range(df_fs_10.shape()):
    print(df_fs_10['name'][row])

## Google Directions
##### Give distance from location to location

In [229]:
# create list from Yelp Top 10 to input as waypoint string for Google Direction API

yelp_list = []

for row in range(df_yelp_10.shape[0]):
    yelp_list.append(str(df_yelp_10['latitude'][row])+'%2C'+str(df_yelp_10['longitude'][row]))

waypoint_string = '|'.join(yelp_list)

In [230]:
g_dir_url = ("https://maps.googleapis.com/maps/api/directions/"
             "json?"
             "origin={1}"
             "&destination={1}"
             "&waypoints=optimize:True|"
             "{2}"
             "&key={0}"
             .format(g_api_key,list[0],waypoint_string))

g_dir_resp = requests.get(g_dir_url)

g_dir_data = g_dir_resp.json()

g_dir_resp.status_code



200

In [231]:
# calculate total distance travelled between all points of interest

time = 0

for x in range(len(g_dir_data['routes'][0]['legs'])):
    time += g_dir_data['routes'][0]['legs'][x]['duration']['value']
                
minutes = round(time/60)


In [232]:
print("\n\nIt will take about {} minutes to drive between each of these locations.".format(minutes))



It will take about 40 minutes to drive between each of these locations.


In [224]:
order = g_dir_data['routes'][0]['waypoint_order']

order_places = []

for i in range(len(order)):
    order_places.append(df_yelp_10['name'][i])

' --> '.join(order_places)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Find places in common with both tables

In [283]:
yelp_names = df_yelp['name']
fs_names = df_fs['name']
common = fs_names[fs_names.isin(yelp_names)]
common

10        Village Ice Cream
53              Studio Bell
61           Ten Foot Henry
63    Rosso Coffee Roasters
Name: name, dtype: object

In [284]:
yelp_names[yelp_names.isin(fs_names)]


0            Ten Foot Henry
1         Village Ice Cream
15              Studio Bell
19    Rosso Coffee Roasters
Name: name, dtype: object

In [291]:
common = common.reset_index(drop=True)
common

0        Village Ice Cream
1              Studio Bell
2           Ten Foot Henry
3    Rosso Coffee Roasters
Name: name, dtype: object

In [293]:
print('\n The top places in common between both sites are:\n')
for row in range(len(common)):
    print('\t',row+1,'-',common[row])


 The top places in common between both sites are:

	 1 - Village Ice Cream
	 2 - Studio Bell
	 3 - Ten Foot Henry
	 4 - Rosso Coffee Roasters


In [311]:
df_fs_10.dtypes

name          object
latitude     float64
longitude    float64
dtype: object

In [312]:
df_yelp_10.dtypes

name          object
latitude     float64
longitude    float64
dtype: object

In [322]:
full_list = pd.concat([df_fs_10,df_yelp_10])
short_list = full_list.sort_index().head(10).reset_index(drop=True)
short_list

,name,latitude,longitude
0,"5 Vines Wine, Craft Beer & Spirits",51.041360,-114.059326
1,Ten Foot Henry,51.041090,-114.065970
2,RiverWalk,51.047774,-114.049147
3,Village Ice Cream,51.042919,-114.054319
4,Sien Lok Park,51.052199,-114.065061
5,Veg-In YYC,51.047396,-114.059111
6,Calgary Court Restaurant,51.051128,-114.061454
7,OEB Breakfast,51.048585,-114.066826
8,Village Ice Cream,51.042734,-114.054295
9,Workshop Kitchen + Culture,51.047090,-114.064890


In [323]:
print("\n\nThe best order to visit the locations is:")
for i in range(len(order)):
    print('   ','  '*i,(short_list['name'][order[i]]))




The best order to visit the locations is:
     5 Vines Wine, Craft Beer & Spirits
       Ten Foot Henry
         RiverWalk
           Village Ice Cream
             Sien Lok Park
               Veg-In YYC
                 Calgary Court Restaurant
                   OEB Breakfast
                     Village Ice Cream
                       Workshop Kitchen + Culture


In [325]:
import os 
import pprint

env_var = os.environ

print("User's Environment variable:") 
pprint.pprint(dict(env_var), width = 1) 

User's Environment variable:
{'CLICOLOR': '1',
 'CONDA_DEFAULT_ENV': 'base',
 'CONDA_EXE': '/Applications/anaconda3/bin/conda',
 'CONDA_PREFIX': '/Applications/anaconda3',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'CONDA_PYTHON_EXE': '/Applications/anaconda3/bin/python',
 'CONDA_SHLVL': '1',
 'GIT_PAGER': 'cat',
 'HOME': '/Users/seanmacdonald',
 'JPY_PARENT_PID': '58466',
 'KERNEL_LAUNCH_TIMEOUT': '40',
 'LANG': 'en_CA.UTF-8',
 'LOGNAME': 'seanmacdonald',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'OLDPWD': '/Users/seanmacdonald',
 'PAGER': 'cat',
 'PATH': '/Applications/anaconda3/bin:/Applications/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin',
 'PROJ_LIB': '/Applications/anaconda3/share/proj',
 'PWD': '/Users/seanmacdonald',
 'SHELL': '/bin/zsh',
 'SHLVL': '1',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.IvuQbpi9yU/Listeners',
 'TERM': 'xterm-color',
 'TERM_PROGRAM': 'Apple_Terminal',
 'TERM_PROGRAM_VERSION': '433',
 'TERM_SESSION_ID': '13A388A1-D5